# Experiment Quick-Start Guide

## Table of Contents

* [Overview of experiment architecture](#Overview-of-experiment-architecture)
* [Experiment workflow](#Experiment-workflow)
    * [Modifying State Variables](#Modifying-State-Variables)
    * [Modifying System Parameters](#Modifying-System-Parameters)
    * [Executing experiments](#Executing-experiments)
    * [Post-processing and analysing results](#Post-processing-and-analysing-results)
    * [Visualizing results](#Visualizing-results)
* [Creating new, customized experiment notebooks](#Creating-new,-customized-experiment-notebooks)
    * Step 1: Select an experiment template
    * Step 2: Create a new notebook
    * Step 3: Customize the experiment
    * Step 4: Execute the experiment
* [Advanced experiment configuration](#Advanced-experiment-configuration)

# Overview of experiment architecture

The experiment architecture is composed of the following four elements - the **model**, **default experiment**, **experiment templates**, and **experiment notebooks**:

1. The **model** is initialized with a default Initial State and set of System Parameters defined in the `model` module.
2. The **default experiment**, in the `experiments.default_experiment` module, is an experiment composed of a single simulation that uses the default cadCAD **model** Initial State and System Parameters. Additional default simulation execution settings such as the number of timesteps and runs are also set in the **default experiment**.
3. The **experiment templates**, in the `experiments.templates` module, contain pre-configured analyses based on the **default experiment**. Examples include `experiments.templates.time_domain_analysis` (simulation in the time-domain over a period of 5 years) and `experiments.templates.eth_price_sweep_analysis` (simulation in the phase-space sweeping over discrete ETH Price values).
4. The **experiment notebooks** perform various scenario analyses by importing existing **experiment templates**, optionally modifying the Initial State and System Parameters within the notebook, and then executing them.

# Experiment workflow

If you just want to run (execute) existing experiment notebooks, simply open the respective notebook and execute all cells.

The experiment notebooks will start by importing some standard dependencies:

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

# External dependencies
import copy
import logging
import numpy as np
import pandas as pd

# Project dependencies
import model.constants as constants
import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import inspect_module

We can then import the default experiment, and create a copy of the simulation object - we create a new copy for each analysis we'd like to perform:

In [ ]:
import experiments.default_experiment as default_experiment
# Or: import experiments.templates.time_domain_analysis as time_domain_analysis

simulation_analysis_1 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_analysis_2 = copy.deepcopy(default_experiment.experiment.simulations[0])

We can use the `inspect_module` method to see the configuration of the default experiment before making changes:

In [ ]:
inspect_module(default_experiment)

## Modifying State Variables

To modify the value of **State Variables** for a specific analysis you need to select the relevant simulation, and update the chosen model Initial State. For example, updating the `eth_supply` Initial State:

In [ ]:
simulation_analysis_1.model.initial_state.update({
    "eth_supply": 100e6
})

## Modifying System Parameters

To modify the value of **System Parameters** for a specific analysis you need to select the relevant simulation, and update the chosen model System Parameter (which is a list of values). For example, updating the `BASE_REWARD_FACTOR` System Parameter:

In [ ]:
simulation_analysis_1.model.params.update({
    "BASE_REWARD_FACTOR": [64, 32]
})

## Executing experiments

We can now execute our custom analysis, and retrieve the post-processed Pandas DataFrame, using the `run(...)` method:

In [ ]:
df, exceptions = run(simulation_analysis_1)

## Post-processing and analysing results

We can see that we had no exceptions for the single simulation we executed:

In [ ]:
exceptions[0]['exception'] == None

We can simply display the Pandas DataFrame to inspect the results:

In [ ]:
df

## Visualizing results

Once we have the results post-processed and in a Pandas DataFrame, we can use Plotly for plotting our results, or Pandas for numerical analyses:

In [ ]:
visualizations.plot_validating_rewards(df, subplot_titles=["Base Reward Factor = 64", "Base Reward Factor = 32"])

# Creating new, customized experiment notebooks

If you want to create an entirely new analysis you'll need to create a new experiment notebook, which entails the following steps:
* Step 1: Select a base experiment template from the [experiments/templates/](../templates/) directory to start from. The template [example_analysis.py](../templates/example_analysis.py) gives an example of extending the default experiment to override default State Variables and System Parameters.
* Step 2: Create a new notebook in [experiments/notebooks/](experiments/notebooks/), using the [template.ipynb](./template.ipynb) notebook as a guide, and import the experiment from the experiment template.
* Step 3: Customize the experiment for your specific analysis.
* Step 4: Execute your experiment, post-process and analyze the results, and create Plotly charts!

# Advanced experiment configuration

### Setting simulation timesteps and unit of time `dt`

In [ ]:
from model.simulation_configuration import TIMESTEPS, DELTA_TIME, SIMULATION_TIME_MONTHS

We can configure the number of simulation timesteps `TIMESTEPS` from a simulation time in months `SIMULATION_TIME_MONTHS`, multiplied by the number of epochs in a month, and divided by the simulation unit of time `DELTA_TIME`:

In [ ]:
SIMULATION_TIME_MONTHS / 12  # Divide months by 12 to get number of years

`DELTA_TIME` is a variable that sets how many epochs are simulated for each timestep. Sometimes if we don't need a finer granularity (1 epoch per timestep, for example), then we can set `DELTA_TIME` to a larger value for better performance. The default value is 1 day or `225` epochs. This means that all our time based states will be for a period of 1 day, which is convenient.

In [ ]:
DELTA_TIME

`TIMESTEPS` is now simply the simulation time in months, multiplied by the number of epochs in a month, divided by `DELTA_TIME`:

```python
TIMESTEPS = constants.epochs_per_month * SIMULATION_TIME_MONTHS // DELTA_TIME
```

In [ ]:
TIMESTEPS

Finally, to set the simulation timesteps (note, you may have to update the environmental processes that depend on the number of timesteps, and override the relevant parameters):

In [ ]:
simulation_analysis_1.timesteps = TIMESTEPS

### Changing the Ethereum network upgrade stage

The model operates over different Ethereum network upgrade stages. The default experiment operates in the "post-merge" Proof of Stake stage.

`Stage` is an Enum, we can import it and see what options we have:

In [ ]:
from model.types import Stage

The model is well documented, and we can view the Python docstring to see what a Stage is, and create a dictionary to view the Enum members:

In [ ]:
print(Stage.__doc__)
{e.name: e.value for e in Stage}

The `PROOF_OF_STAKE` stage, for example, assumes the Beacon Chain has been implemented, EIP1559 has been enabled, and and POW issuance is disabled:

In [ ]:
inspect_module(Stage)

As before, we can update the "stage" System Parameter to set the relevant Stage:

In [ ]:
simulation_analysis_1.model.params.update({
    "stage": [Stage.PROOF_OF_STAKE]
})